In [1]:
import django
import sys, os
import platform
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import timedelta
import numpy as np

if platform.node() == "srv-mcc-apsis":
    sys.path.append("/home/galm/software/django/tmv/BasicBrowser/")
    #sys.path.append('/home/leey/tmv/BasicBrowser/')
else:
    # local paths
    sys.path.append('/home/leey/Documents/Data/tmv/BasicBrowser/')
os.environ.setdefault("DJANGO_SETTINGS_MODULE", "BasicBrowser.settings")
django.setup()

#import from appended path
import scoping.models as sm
import twitter.models as tm
import parliament.models as pm

from django.db.models import Sum, Count
from django.db.models.fields import DateField
from django.db.models.functions import Cast
import textwrap as tw
import datetime
import random

import igraph
#import mpld3

from adjustText import adjust_text

In [2]:
import spacy
from spacy_sentiws import spaCySentiWS

In [3]:
p = sm.Project.objects.get(pk=196)

In [4]:
end_date = datetime.datetime.strptime('2019-09-30-+0000', '%Y-%m-%d-%z')

In [5]:
# Get tweets related to the project
searches = tm.TwitterSearch.objects.filter(project=p)

ttweets = set(tm.Status.objects.filter(
    searches__in=searches,
    text__isnull=False,
    retweeted_status__isnull=True,
    created_at__lte=end_date
).values_list('id',flat=True))

In [6]:
print(len(ttweets))

117235


In [11]:
import random
random.seed(123)

In [12]:
test_texts = tm.Status.objects.filter(
    id__in=random.sample(ttweets, 1000)
).values_list('text',flat=True)

In [26]:
df_test = pd.DataFrame({"text":list(test_texts[0:10])})

In [17]:
from textblob import TextBlob

In [34]:
language = []
for i in df_test.index:
    entry = df_test.loc[i]
    blob = TextBlob(entry["text"])
    language.append(blob.detect_language())
    
language

['de', 'de', 'de', 'de', 'en', 'en', 'es', 'de', 'de', 'de']

In [35]:
df_test["language"] = language

In [36]:
df_test

,text,language
0,Wir stehen hinter jeglichem Friedlichen Protes...,de
1,"#HambacherForst #HambiBleibt ""Als erstes Geric...",de
2,Jetzt erst recht: #Hambibleibt. Wünsche den Be...,de
3,Wir müssen den herrschenden zeigen das wir unb...,de
4,#Hambibleibt https://t.co/xESw54Mh90,en
5,10:07am Uprooting has started at #Hambacher fo...,en
6,@Oaktown1312 informa en directo de la situació...,es
7,"„Wenn Unrecht zu Recht gemacht wird, dann wird...",de
8,Gegen Naturzerstörung für die fossile Gewinnma...,de
9,Klarer Fall von Selbstschutz wie wir es von de...,de


In [37]:
df_test_texts = pd.DataFrame({"text":list(test_texts)})

In [39]:
import time

In [42]:
language = []
for i in df_test_texts.index:
    entry = df_test_texts.loc[i]
    blob = TextBlob(entry["text"])
    language.append(blob.detect_language())
    time.sleep(0.1)
df_test_texts["language"] = language

HTTPError: HTTP Error 429: Too Many Requests

---

In [9]:
texts = tm.Status.objects.filter(
    id__in=ttweets
).values_list('text',flat=True)

In [10]:
texts[0]

'Save the hambach forest from being felled for profit! #myfirstTweet #HambacherForst #HambiBleibt!'

In [ ]:
# spaCySentiWS example
nlp = spacy.load('de')
sentiws = spaCySentiWS(sentiws_path='../SentiWS')
nlp.add_pipe(sentiws)
doc = nlp('Die Dummheit der Unterwerfung blüht in hübschen Farben.')

for token in doc:
    print('{}, {}, {}'.format(token.text, token._.sentiws, token.pos_))